# Service Pattern Boundaries: Ideal vs Worst GTFS Feed

This notebook demonstrates how to generate two theoretical GTFS feeds:
- **Ideal Service:** Every route operates every interval with a 5-minute headway.
- **Worst Service:** Every route operates every interval with the maximum allowed headway (e.g., 120 minutes).

I use these feeds to benchmark mode shares in MATSim under extreme service conditions. They give insights into the limits of the optimisation problem in terms of mode shift potential.

In [4]:
# Step 1: Import and Prepare GTFS Data

from transit_opt.preprocessing.prepare_gtfs import GTFSDataPreparator
from transit_opt.gtfs.gtfs import SolutionConverter
import numpy as np

## Step 2: Configure Paths and Parameters

Set up the GTFS file path, interval length, and allowed headways.  
Make sure 5 and the maximum value (e.g., 120) are included in `allowed_headways`.

In [5]:
gtfs_path = '../data/external/study_area_gtfs_bus.zip'  # Adjust if needed
interval_hours = 6  # 4 intervals per day
allowed_headways = [5, 10, 15, 30, 60, 120]  # Must include 5 and max value

## Step 3: Prepare Optimization Data

Extract optimization data from the GTFS feed.  
This provides route and interval counts, and other metadata needed for solution construction.

In [6]:
preparator = GTFSDataPreparator(
    gtfs_path=gtfs_path,
    interval_hours=interval_hours,
    date=None,
    turnaround_buffer=1.15,
    max_round_trip_minutes=240.0,
    no_service_threshold_minutes=480.0,
    log_level="INFO"
)
opt_data = preparator.extract_optimization_data(allowed_headways)

n_routes = opt_data['n_routes']
n_intervals = opt_data['n_intervals']

2025-09-30 18:19:38,849 - transit_opt.preprocessing.prepare_gtfs - INFO - Initializing GTFSDataPreparator with 6h intervals
2025-09-30 18:19:38,850 - transit_opt.preprocessing.prepare_gtfs - INFO - Loading GTFS feed from ../data/external/study_area_gtfs_bus.zip
2025-09-30 18:19:41,954 - transit_opt.preprocessing.prepare_gtfs - INFO - Using full GTFS feed (all service periods)
2025-09-30 18:19:43,578 - transit_opt.preprocessing.prepare_gtfs - INFO - GTFS loaded and cached in 4.73 seconds
2025-09-30 18:19:43,578 - transit_opt.preprocessing.prepare_gtfs - INFO - Dataset: 13,974 trips, 703,721 stop times
2025-09-30 18:19:43,579 - transit_opt.preprocessing.prepare_gtfs - INFO - Extracting optimization data with 6 allowed headways
2025-09-30 18:19:43,579 - transit_opt.preprocessing.prepare_gtfs - INFO - Extracting route essentials with 6-hour intervals
2025-09-30 18:19:44,900 - transit_opt.preprocessing.prepare_gtfs - WARNING - Route 73302: Round-trip 317.4min exceeds limit (240.0min), filte

## Step 4: Create Ideal and Worst Service Solution Matrices

- **Ideal:** Fill the matrix with the index for 5-minute headway.
- **Worst:** Fill the matrix with the index for the maximum allowed headway.

In [7]:
# Ideal service: 5-minute headway everywhere
ideal_headway = 5
ideal_idx = allowed_headways.index(ideal_headway)
ideal_solution_matrix = np.full((n_routes, n_intervals), ideal_idx, dtype=int)

# Worst service: max headway everywhere
worst_headway = max(allowed_headways)
worst_idx = allowed_headways.index(worst_headway)
worst_solution_matrix = np.full((n_routes, n_intervals), worst_idx, dtype=int)

## Step 5: Convert, Validate, and Generate GTFS Feeds

Use the `SolutionConverter` to validate each solution and generate the corresponding GTFS feed.

In [8]:
converter = SolutionConverter(opt_data)

print("=== IDEAL SERVICE ===")
ideal_validation = converter.validate_solution(ideal_solution_matrix)
print("Valid:", ideal_validation['valid'])
print("Errors:", ideal_validation['errors'])
print("Coverage:", ideal_validation['statistics']['service_percentage'])

ideal_headways_dict = converter.solution_to_headways(ideal_solution_matrix)
ideal_templates = converter.extract_route_templates()

ideal_gtfs_path = converter.build_complete_gtfs(
    ideal_headways_dict,
    ideal_templates,
    service_id='ideal_service',
    output_dir='output/ideal_service_gtfs',
    zip_output=True
)
print("✅ Ideal GTFS feed created at:", ideal_gtfs_path)

=== IDEAL SERVICE ===
Valid: True
Errors: []
Coverage: 100.0
🔍 Processing route 50627...
   ✅ 00-06h: 34.0min template
   ✅ 06-12h: 43.0min template
   ✅ 12-18h: 43.0min template
   ✅ 18-24h: 35.0min template
🔍 Processing route 50628...
   ✅ 00-06h: 65.0min template
   ✅ 06-12h: 79.0min template
   ✅ 12-18h: 80.0min template
   ✅ 18-24h: 64.0min template
🔍 Processing route 50629...
   ✅ 00-06h: 72.0min template
   ✅ 06-12h: 72.0min template
   ✅ 12-18h: 74.0min template
   ✅ 18-24h: 63.0min template
🔍 Processing route 58940...
   ⚠️  00-06h: Using fallback template (30.0min)
   ✅ 06-12h: 20.0min template
   ✅ 12-18h: 30.0min template
   ⚠️  18-24h: Using fallback template (30.0min)
🔍 Processing route 22577...
   ✅ 00-06h: 74.0min template
   ✅ 06-12h: 84.0min template
   ✅ 12-18h: 84.0min template
   ✅ 18-24h: 71.0min template
🔍 Processing route 29092...
   ⚠️  00-06h: Using fallback template (45.0min)
   ✅ 06-12h: 52.0min template
   ✅ 12-18h: 45.0min template
   ⚠️  18-24h: Using fal

Found 155 stops with invalid parent_station references: <StringArray>
[    '450G2617',     '450G7920',     '450G2572',     '450G2393',
     '450G7954',     '450G7613',     '450G6820',     '450G8341',
     '450G7637',     '450G7935',     '450G1142',     '450G7922',
     '450G9423',     '450G9566',     '450G8193',     '029G0052',
  '049GBUSCWY1',    '075G71047',    '079G73001', '109GDDCCBS01',
     '180GCSBS',     '180GMABS',     '180GSHIC',  '269GLC30614',
 '280G00000005',   '330GMA0337',     '339GBB08', '340G00001090',
   '370G100007',   '370G100004',   '370G105120',   '370G100009',
   '380G510101',    '430G00050',    '430G01055',    '430G21031',
   '440GCY0359',     '450G5168',     '450G6011',     '450G7949',
     '450G8049',    '450G21825',    '450G22583',     '450G7924',
     '450G7936',     '450G9478',     '450G8061',     '450G9178',
     '450G8028',  '910GHTRWCBS',  '910GHTRBUS5',     '910GPBRO']
Length: 52, dtype: string


⚠️  Found 155 stops with invalid parent_station references
   Missing parent stations: ['450G2617', '450G7920', '450G2572', '450G2393', '450G7954', '450G7613', '450G6820', '450G8341', '450G7637', '450G7935', '450G1142', '450G7922', '450G9423', '450G9566', '450G8193', '029G0052', '049GBUSCWY1', '075G71047', '079G73001', '109GDDCCBS01', '180GCSBS', '180GMABS', '180GSHIC', '269GLC30614', '280G00000005', '330GMA0337', '339GBB08', '340G00001090', '370G100007', '370G100004', '370G105120', '370G100009', '380G510101', '430G00050', '430G01055', '430G21031', '440GCY0359', '450G5168', '450G6011', '450G7949', '450G8049', '450G21825', '450G22583', '450G7924', '450G7936', '450G9478', '450G8061', '450G9178', '450G8028', '910GHTRWCBS', '910GHTRBUS5', '910GPBRO']
✅ Cleared 155 invalid parent_station references
✅ Fixed and copied stops.txt: 6897 stops
✅ Copied routes.txt: 187 routes
✅ Copied agency.txt: 24 agencies
✅ Generated trips.txt: 37042 trips
✅ Generated stop_times.txt: 1658651 stop times
📅 Using

In [ ]:
print("\n=== WORST SERVICE ===")
worst_validation = converter.validate_solution(worst_solution_matrix)
print("Valid:", worst_validation['valid'])
print("Errors:", worst_validation['errors'])
print("Coverage:", worst_validation['statistics']['service_percentage'])

worst_headways_dict = converter.solution_to_headways(worst_solution_matrix)
worst_templates = converter.extract_route_templates()

worst_gtfs_path = converter.build_complete_gtfs(
    worst_headways_dict,
    worst_templates,
    service_id='worst_service',
    output_dir='output/worst_service_gtfs',
    zip_output=True
)
print("✅ Worst GTFS feed created at:", worst_gtfs_path)


=== WORST SERVICE ===
Valid: True
Errors: []
Coverage: 100.0
🔍 Processing route 50627...
   ✅ 00-06h: 34.0min template
   ✅ 06-12h: 43.0min template
   ✅ 12-18h: 43.0min template
   ✅ 18-24h: 35.0min template
🔍 Processing route 50628...
   ✅ 00-06h: 65.0min template
   ✅ 06-12h: 79.0min template
   ✅ 12-18h: 80.0min template
   ✅ 18-24h: 64.0min template
🔍 Processing route 50629...
   ✅ 00-06h: 72.0min template
   ✅ 06-12h: 72.0min template
   ✅ 12-18h: 74.0min template
   ✅ 18-24h: 63.0min template
🔍 Processing route 58940...
   ⚠️  00-06h: Using fallback template (30.0min)
   ✅ 06-12h: 20.0min template
   ✅ 12-18h: 30.0min template
   ⚠️  18-24h: Using fallback template (30.0min)
🔍 Processing route 22577...
   ✅ 00-06h: 74.0min template
   ✅ 06-12h: 84.0min template
   ✅ 12-18h: 84.0min template
   ✅ 18-24h: 71.0min template
🔍 Processing route 29092...
   ⚠️  00-06h: Using fallback template (45.0min)
   ✅ 06-12h: 52.0min template
   ✅ 12-18h: 45.0min template
   ⚠️  18-24h: Using fa

Found 155 stops with invalid parent_station references: <StringArray>
[    '450G2617',     '450G7920',     '450G2572',     '450G2393',
     '450G7954',     '450G7613',     '450G6820',     '450G8341',
     '450G7637',     '450G7935',     '450G1142',     '450G7922',
     '450G9423',     '450G9566',     '450G8193',     '029G0052',
  '049GBUSCWY1',    '075G71047',    '079G73001', '109GDDCCBS01',
     '180GCSBS',     '180GMABS',     '180GSHIC',  '269GLC30614',
 '280G00000005',   '330GMA0337',     '339GBB08', '340G00001090',
   '370G100007',   '370G100004',   '370G105120',   '370G100009',
   '380G510101',    '430G00050',    '430G01055',    '430G21031',
   '440GCY0359',     '450G5168',     '450G6011',     '450G7949',
     '450G8049',    '450G21825',    '450G22583',     '450G7924',
     '450G7936',     '450G9478',     '450G8061',     '450G9178',
     '450G8028',  '910GHTRWCBS',  '910GHTRBUS5',     '910GPBRO']
Length: 52, dtype: string


⚠️  Found 155 stops with invalid parent_station references
   Missing parent stations: ['450G2617', '450G7920', '450G2572', '450G2393', '450G7954', '450G7613', '450G6820', '450G8341', '450G7637', '450G7935', '450G1142', '450G7922', '450G9423', '450G9566', '450G8193', '029G0052', '049GBUSCWY1', '075G71047', '079G73001', '109GDDCCBS01', '180GCSBS', '180GMABS', '180GSHIC', '269GLC30614', '280G00000005', '330GMA0337', '339GBB08', '340G00001090', '370G100007', '370G100004', '370G105120', '370G100009', '380G510101', '430G00050', '430G01055', '430G21031', '440GCY0359', '450G5168', '450G6011', '450G7949', '450G8049', '450G21825', '450G22583', '450G7924', '450G7936', '450G9478', '450G8061', '450G9178', '450G8028', '910GHTRWCBS', '910GHTRBUS5', '910GPBRO']
✅ Cleared 155 invalid parent_station references
✅ Fixed and copied stops.txt: 6897 stops
✅ Copied routes.txt: 187 routes
✅ Copied agency.txt: 24 agencies
✅ Generated trips.txt: 1764 trips
✅ Generated stop_times.txt: 80991 stop times
📅 Using mi